In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import json
import re
from bs4 import BeautifulSoup as bs

In [33]:
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://www.indeed.com/jobs?q=data+science&l=remote")
time.sleep(1)

soup = bs(driver.page_source, features='html.parser')

In [34]:
results = soup.find(id="jobsearch-Main")

In [35]:
job_elements = soup.find_all("div", class_="slider_item")

In [10]:
basePath = "https://www.indeed.com"
d = {}
# Currently using link_element_id as unique ID.  I am fairly sure it's unique - at least it has been from what I've seen so far.
for job_element in job_elements:
    link_element_id = job_element.find_all("a")[0]["id"]

    d[link_element_id] = {}
    d[link_element_id]["title"] = job_element.find("h2", class_="jobTitle").text.strip()
    d[link_element_id]['company'] = job_element.find("span", class_="companyName").text.strip()
    d[link_element_id]['posted'] = job_element.find("span", class_="date").next_element.next_element.next_element
    try:
        d[link_element_id]['location'] = job_element.find("div", class_="companyLocation").text.strip()
    except AttributeError: 
        d[link_element_id]['location'] = "Not Listed"
    try:
        d[link_element_id]['salary'] = job_element.find("div", class_="metadata").text.strip()
    except AttributeError:
        d[link_element_id]['salary'] = "Not Listed"
    # date_element = job_element.find("div", class_="visually-hidden")  # not useful as-is.  figure out if this is possible to find at all
    time.sleep(0.1)
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="{link_element_id}"]'))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="jobDescriptionText"]')))
    temp_results = results.find("div", id="jobDescriptionText")
    print(d[link_element_id]['title'], d[link_element_id]['company'])
    # print(d)
    # print(temp_results.text.strip())
    d[link_element_id]['desc'] = temp_results.text.strip()

Analytics Data Engineer - Remote Medisolv, Inc.
Machine Learning Engineer - Remote Kforce
Security Research Manager (Data Science) Engtal
Data Collection Projects Argos Multilingual
Associate Engineer, Data Science Nagarro
Staff Data Scientist Honor
Junior Data Analyst Excelgens
Contact Center Data Analyst CharterUP
Data Profiler (Remote) Sumitomo Mitsui Banking Corporation
Data Visualization Specialist Enterra Solutions
Data Science Mentors Chegg
Data Scientist Live Nation
Data Scientist Celink
Data Scientist Matrix New World Engineering
Northeastern University Data Science Co-Op Verizon


In [11]:
d

{'sj_a5f032877fd92e14': {'title': 'Analytics Data Engineer - Remote',
  'company': 'Medisolv, Inc.',
  'posted': 'Posted 30+ days ago',
  'location': 'Remote in Columbia, MD 21044',
  'salary': 'Full-time',
  'desc': 'As an Analytics Data Engineer, you will be responsible for designing, testing, implementing and maintaining data integrations, as well as performing analytic computations that support healthcare quality metrics, risk adjustment, predictive analytics and reporting applications. The role involves collaboration with the data science team and other engineers.\n This platform’s architecture merges data from multiple staging environments in combination with extract files into a single database used for analytic processing. In addition, some web application and BI content development will be required.\n Responsibilities: \n\nAssist in maintaining Medisolv’s cloud-based data infrastructure (primarily Azure SQL Server, Azure Data Factory, Data Lake, and Databricks)\nMaintain and d

In [12]:
import threading

In [38]:
# Testing multithreading

# spawn webdriver for each thread may be required. 

#can instead thread the top part (getting title-->salary) 


basePath = "https://www.indeed.com"
newdict = {}
linkdict = {}
# Currently using link_element_id as unique ID.  I am fairly sure it's unique - at least it has been from what I've seen so far.

def parse_job(job_element, d, idx):
    link_element_id = job_element.find_all("a")[0]["id"]
    linkdict[idx] = link_element_id
    d[link_element_id] = {}
    d[link_element_id]["title"] = job_element.find("h2", class_="jobTitle").text.strip()
    d[link_element_id]['company'] = job_element.find("span", class_="companyName").text.strip()
    d[link_element_id]['posted'] = job_element.find("span", class_="date").next_element.next_element.next_element
    try:
        d[link_element_id]['location'] = job_element.find("div", class_="companyLocation").text.strip()
    except AttributeError: 
        d[link_element_id]['location'] = "Not Listed"
    try:
        d[link_element_id]['salary'] = job_element.find("div", class_="metadata").text.strip()
    except AttributeError:
        d[link_element_id]['salary'] = "Not Listed"
    # date_element = job_element.find("div", class_="visually-hidden")  # not useful as-is.  figure out if this is possible to find at all
    
def get_desc(d, link_id):
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="{link_id}"]'))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="jobDescriptionText"]')))
    time.sleep(1)
    try:
        temp_results = results.find("div", id="jobDescriptionText")
        print(d[link_id]['title'], d[link_id]['company'])
        # print(d)
        # print(temp_results.text.strip())
        d[link_id]['desc'] = temp_results.text.strip()
    except:
        print(f"Prolly empty text: {temp_results}")


threads = []
for idx, job_element in enumerate(job_elements):
    t = threading.Thread(args=(job_element, newdict, idx), target=parse_job) 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()

for link_id in linkdict.values():
    get_desc(newdict, link_id)

Manager, Data Science Indeed
Prolly empty text: None
Security Research Manager (Data Science) Engtal
Prolly empty text: None
Analytics Consultant - Clinical Operations UW Health
Prolly empty text: None
Podcast Producer - Artificial Intelligence TWIML
Prolly empty text: None
Senior AI/ML Data Engineer The Princeton Review
Prolly empty text: None
Vice President of Analytics Delaware North
Prolly empty text: None
Data Collection Projects Argos Multilingual
Prolly empty text: None
Associate Engineer, Data Science Nagarro
Prolly empty text: None
Junior Data Analyst Excelgens
Prolly empty text: None
Staff Data Scientist Honor
Prolly empty text: None
Contact Center Data Analyst CharterUP
Prolly empty text: None
Data Visualization Specialist Enterra Solutions
Prolly empty text: None
Data Scientist Live Nation
Prolly empty text: None
DataOps Engineer II UW Health
Prolly empty text: None
Data Science Mentors Chegg
Prolly empty text: None


In [37]:
newdict

{'sj_5d2d5a79d905d01f': {'title': 'Manager, Data Science',
  'company': 'Indeed',
  'posted': 'Active 4 days ago',
  'location': 'Remote',
  'salary': '$164,000 - $238,000 a year'},
 'sj_885612f252daab9d': {'title': 'Security Research Manager (Data Science)',
  'company': 'Engtal',
  'posted': 'Active 3 days ago',
  'location': 'Remote',
  'salary': '$200,000 - $225,000 a year'},
 'sj_8194593502a6f1bb': {'title': 'Analytics Consultant - Clinical Operations',
  'company': 'UW Health',
  'posted': 'Posted 30+ days ago',
  'location': 'Remote in Madison, WI 53717',
  'salary': '$70,000 - $104,800 a year'},
 'sj_7a3ee9cc2e4a6fae': {'title': 'Podcast Producer - Artificial Intelligence',
  'company': 'TWIML',
  'posted': 'Active 4 days ago',
  'location': 'Remote',
  'salary': 'Estimated $55.3K - $70.1K a year'},
 'sj_596ff55f668905ff': {'title': 'Senior AI/ML Data Engineer',
  'company': 'The Princeton Review',
  'posted': 'Active 1 day ago',
  'location': 'Remote',
  'salary': '$115,000 - 

Once I can get the full job desc, I will want to do the following:
For each job, create a JSON object for it.  Each JSON object will have a structure like

{ "title":job_title, "company":company_name, "location":job_location, "salary":job_salary, "snippet":job_desc_snippet, "desc":full_job_desc, "tags":search_terms_used_to_find_job}

I want a way to go through and if the same job is repeated, add on to the tags in the proper json.  Can do this by instead building it as a dict, appending to the "tags" key so then search_terms_used_to_find_job is instead a list thereof, and the list is appended with the new search terms before being json-ified.

Need to figure out if I can / how to get the date posted.

Need to figure out if the job ID on indeed is unique.  If I can't figure it out, may need my own unique ID for each

Probably remove the job desc snippet since I have full descs now

Need to figure out how to go through all pages

Need to add tags for search params

Can use: https://scrapfly.io/blog/how-to-scrape-indeedcom/
To see how to better optimize this search.  Both for making searching more dynamic as well as using async IO to speed it up